In [1]:
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
from qdev_wrappers.customised_instruments.interfaces.microwave_source_interface import SimulatedMicrowaveSourceInterface, SGS100AMicrowaveSourceInterface

In [2]:
def print_params(instr):
    for k, v in instr.parameters.items():
        print(k, ': ', v())

### Simulated Microwave Source

A simulates microwave source with default values as shown. <br> <br>
NB: status, IQ_state and pulsemod_state can be set with any (reasonable) bool-like argument and the 'raw value' set on the simulated instrument will be 1 or 0 but they will always return booleans as per the on_off_val_mapping in use.  

In [3]:
simulated_source_if = SimulatedMicrowaveSourceInterface('simulated_source')
print_params(simulated_source_if)

IDN :  {'vendor': None, 'model': 'simulated_source', 'serial': None, 'firmware': None}
frequency :  7000000000.0
power :  -10
status :  False
pulsemod_state :  False
IQ_state :  False


In [4]:
simulated_source_if.frequency(6e9)
simulated_source_if.power(-15)
simulated_source_if.status(1)
simulated_source_if.IQ_state(1)
simulated_source_if.pulsemod_state(0)
print_params(simulated_source_if)

IDN :  {'vendor': None, 'model': 'simulated_source', 'serial': None, 'firmware': None}
frequency :  6000000000.0
power :  -15
status :  True
pulsemod_state :  False
IQ_state :  True


Since some sources do not have the IQ state option in hardware this can be entered manually on initialisation and the parameter will not be settable.

In [6]:
simulated_source_if = SimulatedMicrowaveSourceInterface('simulated_source_cw', IQ_option=False)
print_params(simulated_source_if)
try:
    simulated_source_if.IQ_state(1)
except RuntimeError as e:
    print(e)

IDN :  {'vendor': None, 'model': 'simulated_source_cw', 'serial': None, 'firmware': None}
frequency :  7000000000.0
power :  -10
status :  False
pulsemod_state :  False
IQ_state :  False
('Parmeter IQ_state not settable', 'setting simulated_source_cw_IQ_state to 1')


### Real Microwave Source

Now using a real microwave source and checking it works...

In [5]:
instrument = RohdeSchwarz_SGS100A('microwave_source', '')
real_source_if = SGS100AMicrowaveSourceInterface('real_source', instrument)
real_source_if.frequency(6e9)
real_source_if.power(-10)
real_source_if.status(1)
real_source_if.IQ_state(1)
real_source_if.pulsemod_state(0)

ValueError: Could not locate a VISA implementation. Install either the NI binary or pyvisa-py.

In [ ]:
print_params(real_source_if)

Check parameters match those on instrument

In [ ]:
print_params(instrument)